In [ ]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
from keras.utils import np_utils
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import preprocessor as p

In [ ]:
#read dataset jason format and convert into pandas dataframe
df = pd.read_json('data/dataset.json')


In [ ]:
#example of twwet cleam+ning
p.set_options(p.OPT.EMOJI, p.OPT.NUMBER)
p.clean('This is example of tweet emoji reooval #awesome 👍 https://github.com/s/preprocessor')

'This is example of tweet emoji reooval #awesome https://github.com/s/preprocessor'

In [ ]:
#read data and make  a proper format of pandas dataframe
data=df.T.reset_index().drop(['index'], axis=1)
data

,post_id,annotators,rationales,post_tokens
0,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ..."
1,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem..."
2,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]"
3,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[<user>, i, am, bit, confused, coz, chinese, p..."
4,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[this, bitch, in, whataburger, eating, a, burg..."
...,...,...,...,...
20143,9989999_gab,"[{'label': 'offensive', 'annotator_id': 217, '...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...","[if, ur, still, on, twitter, tell, carlton, i,..."
20144,9990225_gab,"[{'label': 'offensive', 'annotator_id': 220, '...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...","[when, i, first, got, on, here, and, said, i, ..."
20145,9991681_gab,"[{'label': 'offensive', 'annotator_id': 206, '...",[],"[was, macht, der, moslem, wenn, der, zion, geg..."
20146,9992513_gab,"[{'label': 'hatespeech', 'annotator_id': 209, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...","[it, is, awful, look, at, world, demographics,..."


In [ ]:
# Text preprocessing
data['post_tokens'] = data['post_tokens'].apply(lambda x: " ".join(x)) # comvert token to a compelte sentence
data['post_tokens'] = data['post_tokens'].apply(lambda x: x.replace("<user>", "")) # remove <user> tag
data['post_tokens']= data['post_tokens'].apply(lambda x: x.replace("TAB_TOKEN", " "))
data['post_tokens'] = data['post_tokens'].apply(lambda x: p.clean(x)) # emoji removal

# Changing label into neumeric value, normal = 0, hatespeech=1, offensive=2
data['annotators'] = data['annotators'].apply((lambda x: x[0]['label']))
data['annotators'] = data['annotators'] .apply((lambda x: re.sub('normal','0',x)))
data['annotators'] = data['annotators'] .apply((lambda x: re.sub('hatespeech','1',x)))
data['annotators'] = data['annotators'] .apply((lambda x: re.sub('offensive','2',x)))
data['annotators'] =pd.to_numeric(data['annotators'])
data

,post_id,annotators,rationales,post_tokens
0,1179055004553900032_twitter,0,[],i dont think im getting my baby them white he ...
1,1179063826874032128_twitter,0,[],we cannot continue calling ourselves feminists...
2,1178793830532956161_twitter,0,[],nawt yall niggers ignoring me
3,1179088797964763136_twitter,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",i am bit confused coz chinese ppl can not acce...
4,1179085312976445440_twitter,1,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",this bitch in whataburger eating a burger with...
...,...,...,...,...
20143,9989999_gab,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",if ur still on twitter tell carlton i said his...
20144,9990225_gab,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",when i first got on here and said i hate trump...
20145,9991681_gab,2,[],was macht der moslem wenn der zion gegen seine...
20146,9992513_gab,1,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",it is awful look at world demographics asians ...


In [ ]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(data['post_tokens'], data['annotators'], test_size=0.05, random_state=42)

In [ ]:
valid_y

9970     1
10389    0
5485     0
5848     0
9671     0
        ..
9653     0
11852    2
18606    1
16782    2
8144     0
Name: annotators, Length: 1008, dtype: int64

In [ ]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  19140
test comments length:  1008


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(train_x)


# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train_x)
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# word level ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(train_x)
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(train_x)
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:506: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
#non deep elarning model
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)   
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [ ]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, WordLevel TF-IDF:   accuracy: 0.5347222222222222     f1 score: 0.4910772067523765
NB, N-Gram Vectors:   accuracy: 0.4126984126984127     f1 score: 0.36137777154590073
NB, CharLevel Vectors:   accuracy: 0.5426587301587301   f1 score: 0.508499491479479


In [ ]:
# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

LR, CharLevel Vectors:   accuracy: 0.5753968253968254   f1 score: 0.561559887926066


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# load the FastTexT pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer
token = text.Tokenizer()
token.fit_on_texts(data['post_tokens'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# convert to one_hot
train_y_onehot = keras.utils.np_utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.np_utils.to_categorical(valid_y, 3)

In [ ]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 10):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [ ]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/10
75/75 [==============================] - 13s 59ms/step - loss: 1.0448 - accuracy: 0.4625
Epoch 2/10
75/75 [==============================] - 5s 68ms/step - loss: 0.9151 - accuracy: 0.5796
Epoch 3/10
75/75 [==============================] - 6s 78ms/step - loss: 0.8760 - accuracy: 0.6098
Epoch 4/10
75/75 [==============================] - 6s 78ms/step - loss: 0.8446 - accuracy: 0.6223
Epoch 5/10
75/75 [==============================] - 6s 78ms/step - loss: 0.8258 - accuracy: 0.6340
Epoch 6/10
75/75 [==============================] - 6s 78ms/step - loss: 0.8052 - accuracy: 0.6462
Epoch 7/10
75/75 [==============================] - 6s 84ms/step - loss: 0.7802 - accuracy: 0.6623
Epoch 8/10
75/75 [==============================] - 6s 83ms/step - loss: 0.7587 - accuracy: 0.6687
Epoch 9/10
75/75 [==============================] - 6s 81ms/step - loss: 0.7188 - accuracy: 0.6932
Epoch 10/10
75/75 [==============================] - 6s 80ms/step - loss: 0.7025 - accuracy: 0.6983
CNN, Wor